In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)



In [9]:
#few-shotを利用する場合

examples = [
{
    "question": "フランス",
    "answer": """
        私はこれを知っています:
        首都: パリ
        言語: フランス語
        食べ物: バゲットとチーズ
        通貨: ユーロ
    """,
},
{
    "question": "イタリア",
    "answer": """
        私はこれを知っています:
        首都: ローマ
        言語: イタリア語
        食べ物: ピザとパスタ
        通貨: ユーロ
    """,
},
{
    "question": "アメリカ",
    "answer": """
        私はこれを知っています:
        首都: ワシントンD.C.
        言語: 英語
        食べ物: ハンバーガーとホットドッグ
        通貨: ドル
    """,    
},
]

In [20]:
class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, input_varibles: PromptTemplate, example: dict):
        self.examples.append(example)

    def select_examples(self, input_varibles: PromptTemplate):
        from random import choice
        return [choice(self.examples)]
    



In [39]:

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

# example_selector = LengthBasedExampleSelector(
#     examples=examples,
#     example_prompt=example_prompt,
#     max_length=100,
# )

example_selector = RandomExampleSelector(
    examples=examples
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: {country}について知っていること?",
    input_variables=["country"],
)

prompt.format(country="日本")



'\n    Human: アメリカ\n    AI: \n        私はこれを知っています:\n        首都: ワシントンD.C.\n        言語: 英語\n        食べ物: ハンバーガーとホットドッグ\n        通貨: ドル\n    \n\n\nHuman: 日本について知っていること?'